Please use the same environment installed from https://github.com/ACCESS-Cloud-Based-InSAR/DockerizedTopsApp

In [1]:
from pathlib import Path
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr

/Users/cmarshak/miniforge3/envs/topsapp_env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [23]:
INDEX_TO_CHECK = 0

In [24]:
nc_paths_new = sorted(list(Path('new_products/').glob('*.nc')))
nc_paths_new

[PosixPath('new_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc'),
 PosixPath('new_products/S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1.nc')]

In [25]:
nc_paths_old = sorted(list(Path('existing_products/').glob('*.nc')))
nc_paths_old

[PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240103-015044-00119W_00033N-PP-629e-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240208_20240115-015044-00119W_00033N-PP-8b54-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240115-015044-00119W_00033N-PP-60cc-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240127-015044-00119W_00033N-PP-4f28-v3_0_1.nc'),
 PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240220_20240208-015044-00119W_00033N-PP-e9c9-v3_0_1.nc')]

In [26]:
new_prod = nc_paths_new[INDEX_TO_CHECK]
new_prod

PosixPath('new_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc')

In [27]:
old_prods = sorted(list(Path('existing_products/').glob(f'{new_prod.name[:38]}*.nc')))
assert len(old_prods) == 1
old_prod = old_prods[0]
old_prod

PosixPath('existing_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc')

In [28]:
with rasterio.open(f'{str(new_prod)}') as ds:
    subdatasets = ds.subdatasets
    print(ds.transform)
subdatasets = sorted([sd for sd in subdatasets if 'productBoundingBox' not in sd])
subdatasets[:2]

| 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


/Users/cmarshak/miniforge3/envs/topsapp_env/lib/python3.11/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


['netcdf:new_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc:/science/grids/corrections/derived/ionosphere/ionosphere',
 'netcdf:new_products/S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1.nc:/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps']

In [29]:
local_vars = [sd.split(':')[-1] for sd in subdatasets]
local_vars[:2]

['/science/grids/corrections/derived/ionosphere/ionosphere',
 '/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps']

In [38]:
ABSOLUTE_DIFF_THRESHOLD = 1e-4

print('New dataset name', new_prod.stem)
print('G(old)en datset name', old_prod.stem)
for golden_path, local_var in zip(subdatasets, local_vars):
    
    print('#'*10)
    print(local_var)
    print('#'*10)
    with rasterio.open(f'netcdf:{str(old_prod)}:{local_var[1:]}') as ds:
        X_golden = ds.read(1)
        trans_golden = ds.transform
    with rasterio.open(f'netcdf:{str(new_prod)}:{local_var[1:]}') as ds:
        X_new = ds.read(1)
        trans_new = ds.transform
        nodata_val = ds.nodata

    if local_var == '/science/grids/corrections/derived/ionosphere/ionosphere':
        iono_new = X_new
        iono_old = X_golden
        

    X_new_data = X_new.ravel()
    X_golden_data = X_golden.ravel()
    print('nodata_val is: ', nodata_val)
    # if np.isnan(nodata_val):
    #     total_mask = ~np.isnan(X_new) & ~np.isnan(X_golden)
    # else:
    #     total_mask = (X_new != nodata_val) & (X_golden != nodata_val)
    total_mask = ~np.isnan(X_new) & ~np.isnan(X_golden)
    if 'connectedComponents' in local_var:
        total_mask = ~np.isin(X_new, [-1, 0])
    if 'amplitude' in local_var:
        total_mask = ~np.isnan(X_new) & (X_new != 0)
    if X_new.dtype in [np.float32, np.float64]:
        X_new_data = X_new[total_mask]
        X_golden_data = X_golden[total_mask]
        rms = np.linalg.norm(X_new_data - X_golden_data)
        bad_pixels_mask = np.abs(X_new_data - X_golden_data) > ABSOLUTE_DIFF_THRESHOLD
        per_pixel_error_large = (bad_pixels_mask).astype(int)
        n_error_large = (per_pixel_error_large).sum()
        perc_error_large = n_error_large / total_mask.sum()

        print(f'The variable {local_var} has RMS error: ', rms)
        print(f'pixels with deviation larger than {ABSOLUTE_DIFF_THRESHOLD}: ', n_error_large)
        print(f'Percent of pixels with deviation larger than {ABSOLUTE_DIFF_THRESHOLD}: ', perc_error_large)
        print(f'Total valid pixels : {total_mask.sum()}')
        if bad_pixels_mask.sum():
            print(f'Mean absolute error (MAE) of bad pixels: {np.nanmean(np.abs(X_new_data - X_golden_data)[bad_pixels_mask])}')

        print(f'The origin of the new dataset is: ', trans_new.c, trans_new.f)
        print(f'The origin of the g(old)en dataset is: ', trans_golden.c, trans_golden.f)
        print('\n')

New dataset name S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1
G(old)en datset name S1-GUNW-A-R-064-tops-20240115_20240103-015044-00119W_00033N-PP-9902-v3_0_1
##########
/science/grids/corrections/derived/ionosphere/ionosphere
##########
nodata_val is:  nan
The variable /science/grids/corrections/derived/ionosphere/ionosphere has RMS error:  0.7747396
pixels with deviation larger than 0.0001:  50238
Percent of pixels with deviation larger than 0.0001:  0.9765760161732403
Total valid pixels : 51443
Mean absolute error (MAE) of bad pixels: 0.0023168539628386497
The origin of the new dataset is:  -119.024167057 34.889267111
The origin of the g(old)en dataset is:  -119.024167057 34.889267111


##########
/science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps
##########
nodata_val is:  nan
The variable /science/grids/corrections/derived/ionosphereBurstRamps/ionosphereBurstRamps has RMS error:  0.57251036
pixels with deviation larger than 0.